In [2]:
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import pandas_udf
import pandas as pd

In [3]:
spark = SparkSession \
          .builder \
          .master('local[*]') \
          .appName('test') \
          .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/08 23:03:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Measurements

In [4]:
measurements_schema = types.StructType([
  types.StructField('day', types.DateType(), nullable=False),
  types.StructField('interval', types.IntegerType(), nullable=False),
  types.StructField('detid', types.StringType(), nullable=False),
  types.StructField('flow', types.IntegerType(), nullable=True),
  types.StructField('occ', types.FloatType(), nullable=True),
  types.StructField('error', types.IntegerType(), nullable=True),
  types.StructField('city', types.StringType(), nullable=False),
  types.StructField('speed', types.FloatType(), nullable=True)
])

In [18]:
measurements_df = spark.read \
                    .option("header", True) \
                    .schema(measurements_schema) \
                    .csv('./data/measurements_test.csv')

In [ ]:
# @pandas_udf(types.TimestampType())
# def get_timestamp(days: pd.Series, intervals: pd.Series) -> pd.Series:
#   dts = pd.to_datetime(days)
#   deltas = pd.to_timedelta(intervals, unit='s')
#   return dts + deltas

In [19]:
@pandas_udf(types.IntegerType())
def get_hour(intervals: pd.Series) -> pd.Series:
  # deltas = pd.to_timedelta(intervals, unit='s')
  # return deltas.dt.components.hours
  return intervals.floordiv(3600)

In [20]:
measurements_df = measurements_df \
                    .withColumn('hour', get_hour('interval')) \
                    .drop('interval')

In [21]:
measurements_df.printSchema()

root
 |-- day: date (nullable = true)
 |-- detid: string (nullable = true)
 |-- flow: integer (nullable = true)
 |-- occ: float (nullable = true)
 |-- error: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- speed: float (nullable = true)
 |-- hour: integer (nullable = true)



## Detectors

In [14]:
detectors_schema = types.StructType([
  types.StructField('detid', types.StringType()),
  types.StructField('length', types.DoubleType()),
  types.StructField('pos', types.DoubleType()),
  types.StructField('fclass', types.StringType()),
  types.StructField('road', types.StringType()),
  types.StructField('limit', types.IntegerType()),
  types.StructField('citycode', types.StringType()),
  types.StructField('lanes', types.IntegerType()),
  types.StructField('linkid', types.IntegerType()),
  types.StructField('long', types.DoubleType()),
  types.StructField('lat', types.DoubleType())
])

In [15]:
detectors_df = spark.read \
                .option("header", True) \
                .schema(detectors_schema) \
                .csv('./data/detectors_public.csv')

In [24]:
df = measurements_df.join(detectors_df, on='detid')

In [28]:
df.count()

999999

## Links

In [31]:
links_schema = types.StructType([
  types.StructField('long', types.DoubleType()),
  types.StructField('lat', types.DoubleType()),
  types.StructField('order', types.IntegerType()),
  types.StructField('piece', types.IntegerType()),
  types.StructField('linkid', types.IntegerType()),
  types.StructField('group', types.FloatType()),
  types.StructField('citycode', types.StringType())
])

In [ ]:
links_df = spark.read \
            .option("header", True) \
            .schema(links_schema) \
            .csv('./data/links.csv')

In [30]:
links_df.show()

+----------+----------+-----+-----+------+-----+--------+
|      long|       lat|order|piece|linkid|group|citycode|
+----------+----------+-----+-----+------+-----+--------+
|10.8910158|48.3610789|    1|    1|     0|  0.1|augsburg|
|10.8908938|48.3609933|    2|    1|     0|  0.1|augsburg|
|10.8906417|48.3608526|    3|    1|     0|  0.1|augsburg|
|10.8904043|48.3607189|    4|    1|     0|  0.1|augsburg|
|10.8899939| 48.360497|    5|    1|     0|  0.1|augsburg|
|10.8909796|48.3611074|    1|    1|     1|  1.1|augsburg|
|10.8908589|48.3610201|    2|    1|     1|  1.1|augsburg|
| 10.890588|48.3608561|    3|    1|     1|  1.1|augsburg|
|10.8903842|48.3607421|    4|    1|     1|  1.1|augsburg|
|10.8901803|48.3606378|    5|    1|     1|  1.1|augsburg|
|10.8899577|48.3605166|    6|    1|     1|  1.1|augsburg|
|10.8935881|48.3648605|    1|    1|     2|  2.1|augsburg|
|10.8935827| 48.364693|    2|    1|     2|  2.1|augsburg|
|10.8934379|48.3644453|    3|    1|     2|  2.1|augsburg|
|10.8928666|48

In [147]:
spark.stop()